In [1]:
import time
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics
from pipeline import create_pipeline
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

In [2]:
# 전처리
# Get Dataset
files={
    'CICI':'/home/irteam/junghye-dcloud-dir/MLAC/new_data/CICI.csv',
    'UNSW':'/home/irteam/junghye-dcloud-dir/MLAC/new_data/UNSW.csv'
}

data = pd.read_csv(files['UNSW'])
data=data[np.isfinite(data).all(1)]

multiclass_labels_1=data['nist_category']

data=data.drop(labels=['label','attack_category','nist_category'],axis=1)

In [3]:
# Define Models
models = []
models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=5, max_features=3)))    
models.append(('CART', DecisionTreeClassifier(max_depth=5)))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=200)))
models.append(('ABoost', AdaBoostClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('MLP', MLPClassifier()))

In [4]:
df=pd.DataFrame(columns=['name','acc','f1_mi','f1_ma','f1_we','rc_mi','rc_ma','rc_we']+\
                 ['pc_mi','pc_ma','pc_we'])
eval_path='/home/irteam/junghye-dcloud-dir/MLAC/evaluation'
confusion_path='/home/irteam/junghye-dcloud-dir/MLAC/confusion_matrix/UNSW_5class'
cnt=0

In [5]:
X_train,X_test,y_train,y_test=train_test_split(data,multiclass_labels_1,test_size=0.3, shuffle=True, stratify=multiclass_labels_1, random_state=34)

In [6]:
# confusion matrix plot
def plot_confusion_matrix(con_mat,labels,title:str,cmap=plt.cm.get_cmap('Blues'),normalize=False):
    plt.imshow(con_mat,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    marks=np.arange(len(labels))
    nlabels=[]
    for k in range(len(con_mat)):
        n=sum(con_mat[k])
        nlabel='{0}(n={1})'.format(labels[k],n)
        nlabels.append(nlabel)

    plt.xticks(marks,labels,rotation=45)
    plt.yticks(marks,nlabels)

    thresh=con_mat.max()/2.
    if normalize:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, '{0}%'.format(con_mat[i, j] * 100 / n), horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    else:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, con_mat[i, j], horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.show()
    #이미지 저장
    plt.savefig(confusion_path+'/'+title+'.png',facecolor='#eeeeee',edgecolor='blue',pad_inches=0.5)
    plt.clf()

In [ ]:
for name, model in models:
    #data_loader()
    # binary classification
    model=create_pipeline(model)
   
    print('training start...',name)
    model.fit(X_train,y_train)
    
    #evaluation
    print('evaluation start...')
    y_pred=model.predict(X_test)
    #evaluation result
    model_eval=[]
    model_eval.append(name)
    
    acc = accuracy_score(y_test, y_pred)
    f1_mi = f1_score(y_test, y_pred,average='micro')
    f1_ma = f1_score(y_test, y_pred,average='macro')
    f1_we = f1_score(y_test, y_pred,average='weighted')
    recall_mi = recall_score(y_test, y_pred, average='micro')
    recall_ma = recall_score(y_test, y_pred, average='macro')
    recall_we = recall_score(y_test, y_pred, average='weighted')
    precision_mi = precision_score(y_test, y_pred, average='micro')
    precision_ma = precision_score(y_test, y_pred, average='macro')
    precision_we = precision_score(y_test, y_pred, average='weighted')
    
    model_eval.extend([acc,f1_mi,f1_ma,f1_we,recall_mi,recall_ma,recall_we,precision_mi,precision_ma,precision_we])
    
    #confusion_metrics
    confusion=metrics.confusion_matrix(y_test,y_pred)
    plot_confusion_matrix(confusion,labels=['Benign','Reconnaissance','Access','DoS','Malware'],title=name)
       


    print(f'name:{name},acc:{acc},f1_score:{f1_mi},{f1_ma},{f1_we},recall:{recall_mi},{recall_ma},{recall_we},precision:{precision_mi},{precision_ma},{precision_we}')
    df.loc[cnt]=model_eval

    cnt+=1
    

df.to_csv(os.path.join(eval_path,'UNSW_5class.csv'),index=False)

training start... RF
evaluation start...
name:RF,acc:0.9565514520889157,f1_score:0.9565514520889157,0.4383597511084513,0.9435144895355523,recall:0.9565514520889157,0.42134829232229654,0.9565514520889157,precision:0.9565514520889157,0.49787980870703513,0.9371965764773339
training start... CART
evaluation start...
name:CART,acc:0.9745757457020464,f1_score:0.9745757457020464,0.6265510493084827,0.9716017611927216,recall:0.9745757457020464,0.6396834721499725,0.9745757457020464,precision:0.9745757457020464,0.6888835645775402,0.9711800311372042
training start... NB
evaluation start...
name:NB,acc:0.8774844091575338,f1_score:0.8774844091575338,0.27510451942168845,0.8262009862436313,recall:0.8774844091575338,0.28609251541885483,0.8774844091575338,precision:0.8774844091575338,0.44868694511112767,0.811388565343739
training start... LDA
evaluation start...
name:LDA,acc:0.9655358935693299,f1_score:0.9655358935693299,0.668456277665882,0.9702376141988761,recall:0.9655358935693299,0.8012411289099891,0

In [2]:
from PIL import Image
import os
confusion_path='/home/irteam/junghye-dcloud-dir/MLAC/confusion_matrix/UNSW_5class'

#파일명
img_files=os.listdir(confusion_path)

images=[] # 이미지 리스트
for img_file in img_files:
    img=Image.open(os.path.join(confusion_path,img_file))
    img.resize((400,300))
    images.append(img)

# 크기
width,height=images[0].size
new_width=width*3
new_height=height*3
new_img=Image.new('RGB',(new_width,new_height))

#이미지 합치기
x_offset=0
y_offset=0
for img in images:
    new_img.paste(img,(x_offset,y_offset))
    x_offset+=width
    if x_offset==new_width:
        x_offset=0
        y_offset+=height


new_img.save(confusion_path+'/'+'merged_image.jpg')